In [1]:
import h5py
import sys
import os

In [25]:
f = h5py.File('input/3.hdf5', 'r')
# List of device groups
deviceList = list(f.keys())
print(deviceList)
# Device 1
device = f[deviceList[0]]
print(device)
# Position dataset of device 1
dataset = device['Position']
print(dataset)
# Dataset of sample 1 of device 1
sample1Dataset = dataset[:, 0, :]
sampleNum = 0
xSum = 0
ySum = 0
zSum = 0
for sample in sample1Dataset:
    sampleNum += 1
    xSum += sample[0]
    ySum += sample[1]
    zSum += sample[2]
print(xSum/sampleNum)
print(ySum/sampleNum)
print(zSum/sampleNum)

['Device_792888129_0']
<HDF5 group "/Device_792888129_0" (9 members)>
<HDF5 dataset "Position": shape (238, 1, 3), type "<f4">
173.57560345305114
-299.29685756138394
-35.91537079690885


In [11]:
shape = dataset.shape
for i in range(shape[1]):
    print(dataset[:,0,:])

[[ 327.9909   -101.216736  -18.436285]
 [ 328.00824  -101.15421   -18.461763]
 [ 327.98926  -101.142365  -18.539389]
 [ 327.9815   -101.11027   -18.48412 ]
 [ 327.98557  -101.04008   -18.537323]
 [ 327.99512  -101.092026  -18.594872]
 [ 328.0345   -101.05515   -18.599087]
 [ 328.0244   -101.14884   -18.679749]
 [ 328.0167   -101.19112   -18.612597]
 [ 328.0634   -101.068634  -18.620121]
 [ 328.05823  -101.127914  -18.602545]
 [ 328.0676   -101.10547   -18.582602]
 [ 328.0468   -101.22104   -18.58321 ]
 [ 328.04285  -101.25061   -18.61305 ]
 [ 328.05383  -101.27011   -18.64544 ]
 [ 328.04254  -101.2928    -18.65487 ]
 [ 328.02234  -101.25197   -18.553913]
 [ 328.00674  -101.25418   -18.53031 ]
 [ 327.98477  -101.26723   -18.502441]
 [ 327.9862   -101.279175  -18.55829 ]
 [ 327.98154  -101.29129   -18.592188]
 [ 327.98358  -101.224884  -18.620909]
 [ 327.9785   -101.22125   -18.590206]
 [ 327.98648  -101.19007   -18.600006]
 [ 328.00983  -101.16718   -18.581606]
 [ 328.0152   -101.112175

In [22]:
class OutputDictionary:
    def __init__(self):
        self.data = {}
    
    def addFile(self, filename):
        self.data[filename] = []
    
    def addDeviceAvg(self, filename, deviceName, avgList):
        deviceDict = {deviceName: avgList}
        self.data[filename].append(deviceDict)
    
    def __str__(self):
        return str(self.data)

In [24]:
nest = OutputDictionary()
nest.addFile("1.hdf5")
nest.addDeviceAvg("1.hdf5", "lights", [23, 54, 65])
nest.addDeviceAvg("1.hdf5", "BAR", [232, 544, 625])
nest.addFile("2.hdf5")
nest.addDeviceAvg("2.hdf5", "arm", [33, 56, 67])
print(nest)

{'1.hdf5': [{'lights': [23, 54, 65]}, {'BAR': [232, 544, 625]}], '2.hdf5': [{'arm': [33, 56, 67]}]}
